## Connect to IDR

In [16]:
from omero.gateway import BlitzGateway
HOST = 'ws://idr.openmicroscopy.org/omero-ws'
conn = BlitzGateway('public', 'public',
                    host=HOST, secure=True)
print(conn.connect())
conn.c.enableKeepAlive(60)

True


## Connect to IDR testing 

In [17]:
HOST = 'ws://idr-testing.openmicroscopy.org/omero-ws'
conn_testing = BlitzGateway('public', 'public',
                    host=HOST, secure=True)
print(conn_testing.connect())
conn_testing.c.enableKeepAlive(60)

True


## Load an image

We load information about the image but **not** the binary data. An image is a 5D-object
The size of a plane is 1024x1024

In [42]:
# For the test an image from idr0067, idr0082, idr0117 should be used.
image_id = 9846309 # idr0067 9846282->idr0082

In [43]:
image = conn.getObject("Image", image_id)
print(image.getName())

S156_HandE.ndpi [0]


In [44]:
image_testing = conn_testing.getObject("Image", image_id)

In [45]:
print("Size Z: %s, Size C: %s, Size T: %s" % (image.getSizeZ(), image.getSizeC(), image.getSizeC()))

Size Z: 1, Size C: 3, Size T: 3


## Load the binary data

To access the binary data, we need to load the `pixels` object from the image and retrieve the plane(s)

In [46]:
def load_plane(image, z, c, t, tile=None):
    pixels = image.getPrimaryPixels()
    if tile is not None:
        print(tile)
        return pixels.getTile(z, c, t, tile)
    return pixels.getPlane(z, c, t)

In [54]:
tile = (0, 0, 5000, 5000)

In [55]:
%%time
# Load from idr
plane = load_plane(image, 0, 0, 0, tile)
print(plane.shape)

(0, 0, 5000, 5000)
(5000, 5000)
CPU times: user 1.81 s, sys: 179 ms, total: 1.99 s
Wall time: 4.21 s


In [56]:
%%time
# Load from idr-testing
plane_testing = load_plane(image_testing, 0, 0, 0, tile)

(0, 0, 5000, 5000)
CPU times: user 1.81 s, sys: 162 ms, total: 1.97 s
Wall time: 4.01 s


To load multiple planes use ``pixels.getPlanes()``

In [10]:
zct_list = [(0, 0, 0), (1, 0, 0)]

In [11]:
def load_planes(image, zct_list):
    pixels = image.getPrimaryPixels()
    return pixels.getPlanes(zct_list)

In [12]:
%%time
# Load from idr
planes = load_planes(image, zct_list)

CPU times: user 233 µs, sys: 26 µs, total: 259 µs
Wall time: 285 µs


In [ ]:
%%time
# Load from idr-testing
planes_testing = load_planes(image_testing, zct_list)

In [5]:
def load_5D_array(image):
    pixels = image.getPrimaryPixels()
    size_z = image.getSizeZ()
    size_c = image.getSizeC()
    size_t = image.getSizeT()
    #z, t, c = 0, 0, 0  # first plane of the image

    zct_list = []
    for t in range(size_t):
        for c in range(size_c):  # all channels
            for z in range(size_z):  # get the Z-stack
                zct_list.append((z, c, t))

    # Load all the planes as YX numpy array
    return pixels.getPlanes(zct_list)

In [6]:
%%time
# Load from idr
planes = load_5D_array(image)

KeyboardInterrupt: 

In [ ]:
%%time
# Load from idr-testing
planes_testing = load_5D_array(image_testing)